In [1]:
import pandas as pd
import numpy as np
#from google.colab import drive
from statsmodels.tools.eval_measures import rmse

In [2]:
#drive.mount('/content/drive')

In [3]:
def df_split(df):
  categoria = []
  for x in df['category_name']:
    categoria.append(x.split('/'))
  return categoria


def sub_categoria(df):  
  lista = df_split(df)
  for y in range(len(lista)):
    lista[y] = lista[y][1]
  return lista

In [4]:
filtro_kids = pd.read_csv('https://github.com/AugustoCRX/bootcamp_blue/blob/main/novos_arquivos_treino/Limpeza%20v3/filtro_Kids_train_v3.csv?raw=true')

In [5]:
filtro_kids['sub_category_name'] = sub_categoria(filtro_kids)

In [6]:
filtro_kids.head()

,name,item_condition_id,category_name,brand_name,price,shipping,item_description,date,stock,date2,day,month,year,description_clear,name_clear,brands_filter,sub_category_name
0,New: Baby K'tan active baby carrier,1,Kids/Gear/Backpacks & Carriers,Other,29.0,1,Brand new in box Size: Medium Color: Coral Ret...,10-7-2018,3,2018-10-07,7,10,2018,brand new box size medium color coral retail r...,new : babi k'tan activ babi carrier,upper_pop_brand,Gear
1,Four puppy dog stuffed animal ty,3,Kids/Toys/Stuffed Animals & Plush,Other,5.0,1,2 beanie babies pugsley wrinkles Puppy with pu...,19-10-2018,27,2018-10-19,19,10,2018,2 beani babi pugsley wrinkl puppi pumpkin big ...,four puppi dog stuf anim ty,basics_brand,Toys
2,Metallic purple slime with sparkles,1,Kids/Toys/Arts & Crafts,Other,8.0,0,"This slime is approximately 1.5 ounces, very s...",12-5-2018,2,2018-12-05,5,12,2018,slime approxim 15 ounc soft kind sticki item a...,metal purpl slime sparkl,basics_brand,Toys
3,Led kids shoes size 3 and size 9,1,Kids/Girls (4+)/Shoes,Other,36.0,0,"Led shoes for kids, seven colors in one shoes,...",1-6-2018,6,2018-01-06,6,1,2018,led shoe kid seven color one shoe red blue gre...,led kid shoe size 3 size 9,upper_pop_brand,Girls (4+)
4,Infantino Baby Carrier,3,Kids/Gear/Backpacks & Carriers,Infantino,7.0,1,Great condition! No stains or tears super easy...,22-10-2018,19,2018-10-22,22,10,2018,great condit stain tear super easi use infanti...,infantino babi carrier,infantino,Gear


In [7]:
filtro_kids['sub_category_name'].value_counts()

Toys                       34947
Girls 0-24 Mos             12126
Girls 2T-5T                11999
Girls (4+)                 10819
Boys 0-24 Mos              10304
Boys (4+)                  10206
Boys 2T-5T                  9444
Feeding                     3437
Diapering                   2758
Gear                        2296
Nursery                     1283
Other                        806
Car Seats & Accessories      736
Pregnancy & Maternity        698
Bathing & Skin Care          589
Health & Baby Care           433
Strollers                    383
Safety                       306
Potty Training               128
Name: sub_category_name, dtype: int64

In [8]:
filtro_kids['item_condition_id'] = filtro_kids['item_condition_id'].replace(5,4)

# Modelo 01
Regressão Linear sem restrição de preço e com variaveis dummies nas variaveis categoricas sub_category_name, item_condition_id (Classificação de 1 a 4), shipping e brands_filter

In [9]:
import statsmodels.api as sm

In [10]:
filtro_kids = pd.get_dummies(filtro_kids, columns=['sub_category_name','brands_filter','item_condition_id','shipping'])

In [11]:
X = filtro_kids.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y = filtro_kids.price
X = sm.add_constant(X)
m1_rl = sm.OLS(exog=X,endog=y)
m1_result = m1_rl.fit()
print(m1_result.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.443
Model:                            OLS   Adj. R-squared:                  0.443
Method:                 Least Squares   F-statistic:                     665.1
Date:                Thu, 13 Oct 2022   Prob (F-statistic):               0.00
Time:                        16:24:32   Log-Likelihood:            -4.9119e+05
No. Observations:              113698   AIC:                         9.827e+05
Df Residuals:                  113561   BIC:                         9.840e+05
Df Model:                         136                                         
Covariance Type:            nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------

In [12]:
filtro_kids['price'].describe()

count    113698.000000
mean         22.027226
std          24.389507
min           3.000000
25%          10.000000
50%          15.000000
75%          25.000000
max         809.000000
Name: price, dtype: float64

In [13]:
m1_ypred = m1_result.predict(X)
m1_RMSE = rmse(y,m1_ypred)
m1_RMSE.round(2)

18.2

# Modelo 02

Regressão Linear com restrição de preço ate 200 dolares e com variaveis dummies nas variaveis categoricas sub_category_name, item_condition_id (Classificação de 1 a 4), shipping e brands_filter

In [14]:
filtro_kids_ajustado = filtro_kids[filtro_kids['price']<=200]

In [15]:
X1 = filtro_kids_ajustado.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y1 = filtro_kids_ajustado.price
X1 = sm.add_constant(X1)
m2_rl = sm.OLS(exog=X1,endog=y1)
m2_result = m2_rl.fit()
print(m2_result.summary())

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.482
Model:                            OLS   Adj. R-squared:                  0.482
Method:                 Least Squares   F-statistic:                     781.7
Date:                Thu, 13 Oct 2022   Prob (F-statistic):               0.00
Time:                        16:24:39   Log-Likelihood:            -4.6908e+05
No. Observations:              113453   AIC:                         9.384e+05
Df Residuals:                  113317   BIC:                         9.397e+05
Df Model:                         135                                         
Covariance Type:            nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------

In [16]:
filtro_kids_ajustado['price'].describe()

count    113453.000000
mean         21.484575
std          21.004194
min           3.000000
25%          10.000000
50%          15.000000
75%          25.000000
max         200.000000
Name: price, dtype: float64

In [17]:
m2_ypred = m2_result.predict(X1)
m2_RMSE = rmse(y1,m2_ypred)
m2_RMSE.round(2)

15.11

# Modelo 03

Arvore de Regressão usando como criterio de seleção dos nós a métrica R²

In [18]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [19]:
X2 = filtro_kids.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y2 = filtro_kids.price
m3_dtr = DecisionTreeRegressor(random_state=10)
result2 = m3_dtr.fit(X2,y2)
m3_r2 = result2.score(X2,y2)
m3_ypred = result2.predict(X2)
m3_mse = mean_squared_error(y2,m3_ypred)
m3_rmse = m3_mse**0.5
print(f"R² = {m3_r2.round(2)}")
print(f"MSE = {m3_mse.round(2)}")
print(f"RMSE = {m3_rmse.round(2)}")

R² = 0.5
MSE = 298.8
RMSE = 17.29


# Modelo 04

Arvore de Regressão com restrição de preço ate 200 dolares e com variaveis dummies nas variaveis categoricas sub_category_name, item_condition_id (Classificação de 1 a 4), shipping e brands_filter

In [20]:
X3 = filtro_kids_ajustado.drop(['name','category_name','item_description','date','stock','date2','day','month','year','stock','price','brand_name','description_clear','name_clear'],axis=1)
y3 = filtro_kids_ajustado.price
m4_dtr = DecisionTreeRegressor(random_state=10)
result3 = m4_dtr.fit(X3,y3)
m4_r2 = result3.score(X3,y3)
m4_ypred = result3.predict(X3)
m4_mse = mean_squared_error(y3,m4_ypred)
m4_rmse = m4_mse**0.5
print(f"R² = {m4_r2.round(2)}")
print(f"MSE = {m4_mse.round(2)}")
print(f"RMSE = {m4_rmse.round(2)}")

R² = 0.53
MSE = 206.99
RMSE = 14.39


# Resultados

| Modelo | R² |RMSE | Observação |
| --- | --- | --- | --- |
| Modelo 01 | 44,3% | 18,20 | Regressão Linear |
| Modelo 02 | 48,2% | 15,11 | Regressão Linear (preço <= 200) |
| Modelo 03 | 50,0% | 17,29 | Arvore Regressão (criterio = R²) |
| Modelo 04 | 53,0% | 14,39 | Arvore Regressão (criterio = R², preço <= 200) |